In [ ]:
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats

In [ ]:
#Functional Class Cutoffs
path_max = 0.689682159032362 
benign_min = 0.807231141721117

sge = '../Data/20250122_BARD1_SGEscores_wAAsub.xlsx'
cc = '/Users/ivan/Downloads/7_genes_for_Starita_02282025_hg38.xlsx'


In [ ]:
def read_data(sge, cc): #Reads all data
    sge = pd.read_excel(sge) #Reads SGE data
    sge = sge.drop(columns = ['R1_score', 'R2_score', 'R3_score']) #Drops replicate score columns
    
    cc = pd.read_excel(cc) #Reads case-control data
    cc = cc[cc['CAVA_GENE'].isin(['BARD1'])] #Filters only for BARD1
    cc = cc[['Classification', '#CHROM', 'REF', 'ALT', 'CAVA_GENE', 'CAVA_CSN', 'CAVA_SO', 'Sample_AAF', 'Sample_ID', 'CaseControl','hg38_start']].copy() #Keeps necessary columns
    print(len(cc))
    return sge, cc

In [ ]:
def classify_vars(sge, path_max, benign_min):
    sge['Classification'] = None
    #sge = sge[sge['Consequence'].isin(['missense_variant'])]
    #sge = sge[~sge['target'].str.contains('X4')]
    
    sge.loc[sge['snv_score'] <= path_max, 'Classification'] = 'NF'
    sge.loc[sge['snv_score'] >= benign_min, 'Classification'] = 'F'
    sge.loc[sge['Classification'] == None, 'Classification'] = 'I'

    sge_nf = sge[sge['Classification'].isin(['NF'])]
    sge_func = sge[sge['Classification'].isin(['F'])]
    
    return sge, sge_func, sge_nf

In [ ]:
def process_cc(cc): #Adds Position ID column to case-control data and splits into cases and controls
    cc['pos_id'] = None #Creates emtpy pos_id column
    cc = cc[cc['ALT'].str.len() == 1].copy()
    cc['hg38_start'] = cc['hg38_start'].astype(str) #Sets hg38 coordinates as str data type
    cc['pos_id'] = cc['hg38_start'] + ':' + cc['ALT'] #Creates position ID

    cc_control = cc[cc['CaseControl'].isin(['Control'])] #Creates control df
    cc_case = cc[cc['CaseControl'].isin(['Case'])] #creates case df

    return cc, cc_control, cc_case

In [ ]:
def analyze(cc, sge):
    merged = pd.merge(cc, sge, on = 'pos_id', how = 'inner')
    merged = merged.dropna(subset = ['Classification_y'])

    contingency_tab = merged[['CaseControl', 'Classification_y']]

    contingency_tab = pd.crosstab(merged['CaseControl'], merged['Classification_y'])
    #merged.to_excel('/Users/ivan/Downloads/casecontrol_test2.xlsx')


    table_array = contingency_tab.values
    oddsratio, p_value = stats.fisher_exact(table_array)

    display(contingency_tab)
    print(f"Odds Ratio: {1/oddsratio}")
    print(f"P-value: {p_value}")

In [ ]:
def main():
    sge_df, cc_df = read_data(sge, cc)
    sge_all,sge_func, sge_nf = classify_vars(sge_df, path_max, benign_min)
    cc_all, controls, cases = process_cc(cc_df)
    #analyze(controls, cases, sge_func, sge_nf)
    analyze(cc_all, sge_all)

In [ ]:
main()